In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import cv2
import zipfile as zp
import glob

In [ ]:
input_path = "/kaggle/input/aerial-cactus-identification"
## train
train_zpfile = zp.ZipFile(os.path.join(input_path, "train.zip"))
train_zpfile.extractall("/kaggle/working")
## test
test_zpfile = zp.ZipFile(os.path.join(input_path, "test.zip"))
test_zpfile.extractall("/kaggle/working")

In [ ]:
file = pd.read_csv('../input/aerial-cactus-identification/train.csv')
file.columns

In [ ]:
img=[]
for i in range(17500):
    temp=cv2.imread('/kaggle/working/train/'+file['id'][i],cv2.IMREAD_COLOR)
    temp=temp.reshape(32*32*3)
    img.insert(i,temp)

In [ ]:
x_base,y_base=img[:17500],file['has_cactus'][:17500]

x_train,y_train=x_base[:4000],y_base[:4000]
x_validation,y_validation=x_base[15000:17500],y_base[15000:17500]

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import make_pipeline

In [ ]:
anova_filter = SelectKBest(f_classif, k=32*32*2)
svc = SVC(kernel='rbf',C=5,probability=True)
anova_svm = make_pipeline(anova_filter, svc)


anova_svm.fit(x_base[:15000], y_base[:15000])
print("show svc mod.")
pred = anova_svm.predict(x_validation)
report = classification_report(y_validation, pred)
confusion_mat = confusion_matrix(y_validation, pred)
print(report)                                                                                                                                          
print(confusion_mat)  

In [ ]:
file_test = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
file_test.columns

In [ ]:
test=[]
for i in range(4000):
    temp=cv2.imread('/kaggle/working/test/'+file_test['id'][i],cv2.IMREAD_COLOR)
    temp=temp.reshape(32*32*3)
    test.insert(i,temp)

In [ ]:
pred=anova_svm.predict(test)
file_test['has_cactus']=pred
file_test.to_csv('/kaggle/working/submission.csv',index=False)
file_test.head(10)